# DROP

DROP (Discrete Reasoning Over Paragraphs) เป็นเกณฑ์มาตรฐานที่ออกแบบมาเพื่อประเมินความสามารถในการให้เหตุผลขั้นสูงของโมเดลภาษา ผ่านงานตอบคำถามที่ซับซ้อน ซึ่งประกอบด้วยความท้าทายมากกว่า 9500 ข้อ ที่ต้องใช้การคำนวณตัวเลข การให้เหตุผลหลายขั้นตอน และการตีความข้อมูลจากข้อความ

### Arguments
There are two optional arguments when using the DROP benchmark:

- [Optional] tasks: a list of tasks (DROPTask enums), which specifies the subject areas for model evaluation. By default, this is set to all tasks. The list of DROPTask enums can be found here.
- [Optional] n_shots: the number of "shots" to use for few-shot learning. This is set to 5 by default and cannot exceed 5.

In [ ]:
from langchain_openai import AzureChatOpenAI
from deepeval.models.base_model import DeepEvalBaseLLM
import sys
sys.path.append('/opt/project/src/evaluate_llm/')
from api_key_config import settings
import os

os.environ["OPENAI_API_VERSION"] = settings.OPENAI_API_VERSION
os.environ["OPENAI_API_KEY"] = settings.OPENAI_API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = settings.AZURE_OPENAI_ENDPOINT

class AzureOpenAI(DeepEvalBaseLLM):
    def __init__(
        self,
        model
    ):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        return chat_model.invoke(prompt).content

    async def a_generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        res = await chat_model.ainvoke(prompt)
        return res.content

    def get_model_name(self):
        return "Custom Azure OpenAI Model"

# Replace these with real values
custom_model = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
)
azure_openai = AzureOpenAI(model=custom_model)

In [ ]:
from deepeval.benchmarks import DROP
from deepeval.benchmarks.tasks import DROPTask

# Define benchmark with specific tasks and shots
benchmark = DROP(
    tasks=[DROPTask.HISTORY_1002, DROPTask.NFL_649],
    n_shots=3
)

# Replace 'mistral_7b' with your own custom model
benchmark.evaluate(model=mistral_7b)
print(benchmark.overall_score)